# TSFS12 Hand-in exercise 1: Discrete planning in structured road networks
Erik Frisk (erik.frisk@liu.se)

Do initial imports of packages needed

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from misc import Timer, LatLongDistance
from queues import FIFO, LIFO, PriorityQueue
from osm import loadOSMmap

Activate plots in external windows (needed for mission definition)

In [2]:
%matplotlib

Using matplotlib backend: TkAgg


# Load map

In [3]:
dataDir = '' #linux dataDir = '../Maps/' 
osmFile = 'linkoping.osm'
figFile = 'linkoping.png'
osmMap = loadOSMmap(dataDir + osmFile, dataDir + figFile)

num_nodes = len(osmMap.nodes)  # Number of nodes in the map

Define function to compute possible next nodes from node x and the corresponding distances distances.

In [4]:
def f_next(x):
    """Compute, neighbours for a node"""
    cx = osmMap.distancematrix[x, :].tocoo()
    return cx.col, np.full(cx.col.shape, np.nan), cx.data

# Display basic information about map

Print some basic map information

In [5]:
osmMap.info()

OSMFile: ../Maps/linkoping.osm
Number of nodes: 12112
Number of roads: 2977
Map bounds
  Lon: 15.572100 - 15.650100
  Lat: 58.391000 - 58.420200
  Size: 4545.8 x 3246.9 (m)
Figure file: ../Maps/linkoping.png
  Size: 2296 x 1637 (px)


Plot the map

In [6]:
#plt.figure(10)
osmMap.plotmap()
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Linköping');

Which nodes are neighbors to node with index 110?

In [7]:
n, _, d = f_next(110)
print(f'Neighbours: {n}')
print(f'Distances: {d}')

Neighbours: [3400 3401]
Distances: [8.51886025 8.7868951 ]


Look up the distance (in meters) between the nodes 110 and 3400 in the distance matrix?

In [8]:
print(osmMap.distancematrix[110, 3400])

8.518860245357462


Latitude and longitude of node 110

In [9]:
p = osmMap.nodeposition[110]
print(f'Longitude = {p[0]:.3f}, Latitude = {p[1]:.3f}')

Longitude = 15.574, Latitude = 58.398


Plot the distance matrix and illustrate sparseness

In [10]:
#plt.figure(20)
plt.spy(osmMap.distancematrix>0, markersize=0.5)
plt.xlabel('Node index')
plt.ylabel('Node index')
density = np.sum(osmMap.distancematrix>0)/num_nodes**2
_ = plt.title(f'Density {density*100:.2f}%')

# Define planning mission

In the map, click on start and goal positions to define a mission. Try different missions, ranging from easy to more complex. 

An easy mission is a mission in the city centre; while a more difficult could be from Vallastaden to Tannefors.

In [11]:
plt.figure(30, clear=True)
osmMap.plotmap()
plt.title('Linköping - click in map to define mission')
mission = {}

mission['start'] = osmMap.getmapposition()
plt.plot(mission['start']['pos'][0], mission['start']['pos'][1], 'bx')
mission['goal'] = osmMap.getmapposition()
plt.plot(mission['goal']['pos'][0], mission['goal']['pos'][1], 'bx')

plt.xlabel('Longitude')
plt.ylabel('Latitude')

print('Mission: Go from node %d ' % (mission['start']['id']), end='')
if mission['start']['name'] != '':
    print('(' + mission['start']['name'] + ')', end='')
print(' to node %d ' % (mission['goal']['id']), end='')
if mission['goal']['name'] != '':
    print('(' + mission['goal']['name'] + ')', end='')
print('')

Mission: Go from node 20 (Gumpekullavägen) to node 160 (Hans Meijers väg)


In [12]:
mission

{'start': {'id': 20,
  'pos': (15.636418548387098, 58.41856785826736),
  'name': 'Gumpekullavägen'},
 'goal': {'id': 160,
  'pos': (15.577604032258066, 58.400442516343475),
  'name': 'Hans Meijers väg'}}

Some pre-defined missions to experiment with. To use the first pre-defined mission, call the planner with
```planner(num_nodes, pre_mission[0], f_next, cost_to_go)```.

In [13]:
pre_mission = [
    {'start': {'id': 10906}, 'goal': {'id': 1024}},
    {'start': {'id': 3987}, 'goal': {'id': 4724}},
    {'start': {'id': 423}, 'goal': {'id': 364}}]
# mission = pre_mission[0]

# Implement planners

In [14]:
def DepthFirst(num_nodes, mission, f_next, heuristic=None, num_controls=0):
    """Depth first planner."""
    t = Timer()
    t.tic()
    
    unvis_node = -1
    previous = np.full(num_nodes, dtype=np.int, fill_value=unvis_node)
    cost_to_come = np.zeros(num_nodes)
    control_to_come = np.zeros((num_nodes, num_controls), dtype=np.int)

    startNode = mission['start']['id']
    goalNode = mission['goal']['id']

    q = LIFO()
    q.insert(startNode)
    foundPlan = False

    while not q.IsEmpty():
        x = q.pop()
        if x == goalNode:
            foundPlan = True
            break
        neighbours, u, d = f_next(x)
        for xi, ui, di in zip(neighbours, u, d):
            if previous[xi] == unvis_node:
                previous[xi] = x
                q.insert(xi)
                cost_to_come[xi] = cost_to_come[x] + di
                if num_controls > 0:
                    control_to_come[xi] = ui

    # Recreate the plan by traversing previous from goal node
    if not foundPlan:
        return []
    else:
        plan = [goalNode]
        length = cost_to_come[goalNode]
        control = []
        while plan[0] != startNode:
            if num_controls > 0:
                control.insert(0, control_to_come[plan[0]])
            plan.insert(0, previous[plan[0]])

        return {'plan': plan,
                'length': length,
                'num_visited_nodes': np.sum(previous != unvis_node),
                'name': 'DepthFirst',
                'time': t.toc(),
                'control': control,
                'visited_nodes': previous[previous != unvis_node]}

## Planning example using the DepthFirst planner

Make a plan using the ```DepthFirst``` planner

In [15]:
df_plan = DepthFirst(num_nodes, mission, f_next)
print('{:.1f} m, {} visited nodes, planning time {:.1f} msek'.format(
    df_plan['length'],
    df_plan['num_visited_nodes'],
    df_plan['time']*1e3))

26631.0 m, 6232 visited nodes, planning time 900.0 msek


Plot the resulting plan

In [16]:
#plt.figure(40, clear=True)
osmMap.plotmap()
osmMap.plotplan(df_plan['plan'], 'b',
                label=f"Depth first ({df_plan['length']:.1f} m)")
plt.title('Linköping')
_ = plt.legend()

Plot nodes visited during search

In [17]:
#plt.figure(41, clear=True)
osmMap.plotmap()
osmMap.plotplan(df_plan['visited_nodes'], 'b.')
plt.ylabel('Latitude')
plt.xlabel('Longitude')
_ = plt.title('Nodes visited during DepthFirst search')

Names of roads along the plan ...

In [18]:
planWayNames = osmMap.getplanwaynames(df_plan['plan'])
print('Start: ', end='')
for w in planWayNames[:-1]:
    print(w + ' -- ', end='')
print('Goal: ' + planWayNames[-1])

Start: Gumpekullavägen -- Åsbjörnsgatan -- Vigfastgatan -- Gumpekullavägen -- Anders Ljungstedts gata -- Norrköpingsvägen -- Tullrondellen -- Nya Tanneforsvägen -- Drottningrondellen -- Nya Tanneforsvägen -- Drottninggatan -- Hamngatan -- Wavrinskys gränd -- Snickaregatan -- Drottninggatan -- Hospitalsgränd -- Nygatan -- Apotekaregatan -- Ågatan -- Platensgatan -- Teatergatan -- Platensgatan -- Vasavägen -- Järnvägsavenyn -- Järnvägsgatan -- Industrigatan -- Järnvägsgatan -- Industrigatan -- Steningerondellen -- Kallerstadsleden -- Industrigatan -- Mellangatan -- Industrigatan -- Bergsvägen -- Bergsrondellen -- Bergsvägen -- Industrigatan -- Bergsrondellen -- Bergsvägen -- Bergsrondellen -- Industrigatan -- Danmarksgatan -- Banérgatan -- Gustav Adolfsgatan -- Vallgatan -- Götgatan -- Abiskorondellen -- Vasavägen -- Abiskorondellen -- Västra vägen -- Västanågatan -- Bjälbogatan -- Ulvåsavägen -- Majgatan -- Malmslättsvägen -- Östgötagatan -- Brunnsgatan -- Gröngatan -- Västra vägen -- S

# Define planners

Here, write your code for your planners. Start with the template code for the depth first search and extend.

In [19]:
def BreadthFirst(num_nodes, mission, f_next, heuristic=None, num_controls=0):
    """Breadth first planner."""
    t = Timer()
    t.tic()
    
    unvis_node = -1
    previous = np.full(num_nodes, dtype=np.int, fill_value=unvis_node)
    cost_to_come = np.zeros(num_nodes)
    control_to_come = np.zeros((num_nodes, num_controls), dtype=np.int)

    startNode = mission['start']['id']
    goalNode = mission['goal']['id']

    #First in, First Out from Queues
    q = FIFO()
    q.insert(startNode)
    foundPlan = False

    while not q.IsEmpty():
        x = q.pop()
        if x == goalNode:
            foundPlan = True
            break
        neighbours, u, d = f_next(x)
        for xi, ui, di in zip(neighbours, u, d):
            if previous[xi] == unvis_node:
                previous[xi] = x
                q.insert(xi)
                cost_to_come[xi] = cost_to_come[x] + di
                if num_controls > 0:
                    control_to_come[xi] = ui

    # Recreate the plan by traversing previous from goal node
    if not foundPlan:
        return []
    else:
        plan = [goalNode]
        length = cost_to_come[goalNode]
        control = []
        while plan[0] != startNode:
            if num_controls > 0:
                control.insert(0, control_to_come[plan[0]])
            plan.insert(0, previous[plan[0]])

        return {'plan': plan,
                'length': length,
                'num_visited_nodes': np.sum(previous != unvis_node),
                'name': 'Breadth First',
                'time': t.toc(),
                'control': control,
                'visited_nodes': previous[previous != unvis_node]}

def Dijkstra(num_nodes, mission, f_next, heuristic=None, num_controls=0):
    """Djikstra planner."""
    t = Timer()
    t.tic()
    
    unvis_node = -1
    previous = np.full(num_nodes, dtype=np.int, fill_value=unvis_node)
    cost_to_come = np.zeros(num_nodes)
    control_to_come = np.zeros((num_nodes, num_controls), dtype=np.int)

    startNode = mission['start']['id']
    goalNode = mission['goal']['id']

    #Priority queue based on the lower cost-to-go
    q = PriorityQueue()
    q.insert(0,startNode)
    foundPlan = False

    while not q.IsEmpty():
        x_ctc = q.pop()
        x = x_ctc[1]
        if x == goalNode:
            foundPlan = True
            break
        neighbours, u, d = f_next(x)
        for xi, ui, di in zip(neighbours, u, d):
            if previous[xi] == unvis_node or cost_to_come[xi] > cost_to_come[x] + di:
                previous[xi] = x
                cost_to_come[xi] = cost_to_come[x] + di
                q.insert(cost_to_come[xi],xi)
                if num_controls > 0:
                    control_to_come[xi] = ui

    # Recreate the plan by traversing previous from goal node
    if not foundPlan:
        return []
    else:
        plan = [goalNode]
        length = cost_to_come[goalNode]
        control = []
        while plan[0] != startNode:
            if num_controls > 0:
                control.insert(0, control_to_come[plan[0]])
            plan.insert(0, previous[plan[0]])

        return {'plan': plan,
                'length': length,
                'num_visited_nodes': np.sum(previous != unvis_node),
                'name': 'Djikstra',
                'time': t.toc(),
                'control': control,
                'visited_nodes': previous[previous != unvis_node]}


def Astar(num_nodes, mission, f_next, h, num_controls=0):
    """Astar planner."""
    t = Timer()
    t.tic()
    unvis_node = -1
    previous = np.full(num_nodes, dtype=np.int, fill_value=unvis_node)
    cost_to_come = np.zeros(num_nodes)
    control_to_come = np.zeros((num_nodes, num_controls), dtype=np.int)
    startNode = mission['start']['id']
    goalNode = mission['goal']['id']
    q = PriorityQueue()
    q.insert(0+ h(startNode),startNode)
    foundPlan = False

    while not q.IsEmpty():
        x_ctc = q.pop()
        x = x_ctc[1]
        if x == goalNode:
            foundPlan = True
            break

        neighbours, u, d = f_next(x)
        for xi, ui, di in zip(neighbours, u, d):
            if previous[xi] == unvis_node or cost_to_come[xi] > cost_to_come[x] + di:
                previous[xi] = x
                cost_to_come[xi] = cost_to_come[x] + di
                q.insert(cost_to_come[xi]+h(xi),xi)
                if num_controls > 0:
                    control_to_come[xi] = ui

    # Recreate the plan by traversing previous from goal node
    if not foundPlan:
        return []
    else:
        plan = [goalNode]
        length = cost_to_come[goalNode]
        control = []
        while plan[0] != startNode:
            if num_controls > 0:
                control.insert(0, control_to_come[plan[0]])
            plan.insert(0, previous[plan[0]])

        return {'plan': plan,
                'length': length,
                'num_visited_nodes': np.sum(previous != unvis_node),
                'name': 'Astar',
                'time': t.toc(),
                'control': control,
                'visited_nodes': previous[previous != unvis_node]}




def BestFirst(num_nodes, mission, f_next, h, num_controls=0):
    """BestFirst planner."""
    t = Timer()
    t.tic()
    unvis_node = -1
    previous = np.full(num_nodes, dtype=np.int, fill_value=unvis_node)
    cost_to_come = np.zeros(num_nodes)
    control_to_come = np.zeros((num_nodes, num_controls), dtype=np.int)
    startNode = mission['start']['id']
    goalNode = mission['goal']['id']
    q = PriorityQueue()
    q.insert(h(startNode),startNode)
    foundPlan = False

    while not q.IsEmpty():
        x_ctc = q.pop()
        x = x_ctc[1]
        if x == goalNode:
            foundPlan = True
            break

        neighbours, u, d = f_next(x)
        for xi, ui, di in zip(neighbours, u, d):
            if previous[xi] == unvis_node:
                previous[xi] = x
                cost_to_come[xi] = cost_to_come[x] + di
                q.insert(h(xi),xi)
                if num_controls > 0:
                    control_to_come[xi] = ui

    # Recreate the plan by traversing previous from goal node
    if not foundPlan:
        return []
    else:
        plan = [goalNode]
        length = cost_to_come[goalNode]
        control = []
        while plan[0] != startNode:
            if num_controls > 0:
                control.insert(0, control_to_come[plan[0]])
            plan.insert(0, previous[plan[0]])

        return {'plan': plan,
                'length': length,
                'num_visited_nodes': np.sum(previous != unvis_node),
                'name': 'Astar',
                'time': t.toc(),
                'control': control,
                'visited_nodes': previous[previous != unvis_node]}




# Define heuristic for Astar and BestFirst planners

Define the heuristic for Astar and BestFirst. The ```LatLongDistance``` function will be useful.

In [20]:
def cost_to_go(x, xg):
    p_x = osmMap.nodeposition[x]
    p_g = osmMap.nodeposition[xg]
    return 0.0;

#heuristic function
def h(x):
    goalNode = mission['goal']['id']
    p_x = osmMap.nodeposition[x]
    p_g = osmMap.nodeposition[goalNode]
    return LatLongDistance(p_x, p_g)

# Investigations using all planners

Comparing the planners

In [21]:
#df_planDF = DepthFirst(num_nodes, pre_mission[1], f_next)
df_planBF = BreadthFirst(num_nodes, mission, f_next)
df_planD = Dijkstra(num_nodes, mission, f_next)
df_planA = Astar(num_nodes, mission, f_next, h)
df_planBestF = BestFirst(num_nodes, mission, f_next, h)


plt.figure(100, clear=True)
osmMap.plotmap()
osmMap.plotplan(df_planBF['plan'], 'r',
                label=f"Breadth first ({df_planBF['length']:.1f} m - {df_planBF['num_visited_nodes']} visited nodes - {df_planBF['time']*1e3:.1f} ms)")
osmMap.plotplan(df_planD['plan'], 'g',
                label=f"Dijkstra ({df_planD['length']:.1f} m- {df_planD['num_visited_nodes']} visited nodes - {df_planD['time']*1e3:.1f} ms)")
osmMap.plotplan(df_planA['plan'], 'y',
                label=f"Astar ({df_planA['length']:.1f} m- {df_planA['num_visited_nodes']} visited nodes - {df_planA['time']*1e3:.1f} ms)")
osmMap.plotplan(df_planBestF['plan'], 'b',
                label=f"Best First ({df_planBestF['length']:.1f} m- {df_planBestF['num_visited_nodes']} visited nodes - {df_planBestF['time']*1e3:.1f} ms)")
plt.title('Linköping')
_ = plt.legend()

Details for BreadthFirst

In [22]:
print('{:.1f} m, {} visited nodes, planning time {:.1f} msek'.format(
    df_planBF['length'],
    df_planBF['num_visited_nodes'],
    df_planBF['time']*1e3))

plt.figure(101, clear=True)
osmMap.plotmap()
osmMap.plotplan(df_planBF['visited_nodes'], 'b.')
plt.ylabel('Latitude')
plt.xlabel('Longitude')
_ = plt.title('Nodes visited during BreadthFirst search')

5670.7 m, 11663 visited nodes, planning time 1930.0 msek


Details for Dijkstra

In [23]:
print('{:.1f} m, {} visited nodes, planning time {:.1f} msek'.format(
    df_planD['length'],
    df_planD['num_visited_nodes'],
    df_planD['time']*1e3))

plt.figure(102, clear=True)
osmMap.plotmap()
osmMap.plotplan(df_planD['visited_nodes'], 'b.')
plt.ylabel('Latitude')
plt.xlabel('Longitude')
_ = plt.title('Nodes visited during Dijkstra search')

5600.1 m, 11294 visited nodes, planning time 2019.1 msek


Details for Astar

In [24]:
print('{:.1f} m, {} visited nodes, planning time {:.1f} msek'.format(
    df_planA['length'],
    df_planA['num_visited_nodes'],
    df_planA['time']*1e3))

plt.figure(103, clear=True)
osmMap.plotmap()
osmMap.plotplan(df_planA['visited_nodes'], 'b.')
plt.ylabel('Latitude')
plt.xlabel('Longitude')
_ = plt.title('Nodes visited during Astar search')

5600.1 m, 7195 visited nodes, planning time 1261.0 msek


Details for BestFirst

In [25]:
print('{:.1f} m, {} visited nodes, planning time {:.1f} msek'.format(
    df_planBestF['length'],
    df_planBestF['num_visited_nodes'],
    df_planBestF['time']*1e3))

plt.figure(104, clear=True)
osmMap.plotmap()
osmMap.plotplan(df_planBestF['visited_nodes'], 'b.')
plt.ylabel('Latitude')
plt.xlabel('Longitude')
_ = plt.title('Nodes visited during BestFirst search')

6256.6 m, 788 visited nodes, planning time 140.0 msek
